In [7]:
import torch.nn as nn
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score as R2
from sklearn.metrics import mean_squared_error as MSE
from sklearn.multioutput import MultiOutputRegressor
import numpy as np
import torch
import matplotlib.pyplot as plt
import os
import time
import torch.optim as optim
import torch.nn.functional as F
from functools import reduce

### Knowledge guided machine learning implementation using gated recurrent units
### Inspired from https://gmd.copernicus.org/articles/15/2839/2022

In [8]:
class KGML(nn.Module):
    def __init__(self, ninp, nhid, nlayers, nout, dropout):
        super(KGML, self).__init__()
        if nlayers > 1:
            self.gru = nn.GRU(ninp, nhid,nlayers,dropout=dropout)
        else:
            self.gru = nn.GRU(ninp, nhid,nlayers)
        #self.densor1 = nn.ReLU() #can test other function
        self.densor2 = nn.Linear(nhid, nout)
        self.nhid = nhid
        self.nlayers = nlayers
        self.drop=nn.Dropout(dropout)
        self.init_weights()

    def init_weights(self):
        initrange = 0.1 #may change to a small value
        self.densor2.bias.data.zero_()
        self.densor2.weight.data.uniform_(-initrange, initrange)

    def forward(self, inputs, hidden):
        output, hidden = self.gru(inputs, hidden)
        #output = self.densor1(self.drop(output))
        #output = torch.exp(self.densor2(self.drop(output))) # add exp
        output = self.densor2(self.drop(output)) # add exp
        return output, hidden

#bsz should be batch size
    def init_hidden(self, bsz):
        weight = next(self.parameters())
        weight = weight.float()
        return weight.new_zeros(self.nlayers, bsz, self.nhid)
    
def get_ini(x,ind,nout):
    initials=[]
    for i in range(len(ind)):
        initials.append(x[:,:,ind[i]].view(x.size(0),x.size(1),nout[i]))
    return initials

def myloss_mul_sum(output, target,loss_weights):
    loss = 0.0
    nout=output.size(2)
    for i in range(nout):
        loss = loss + loss_weights[i]*torch.mean((output[:,:,i] - target[:,:,i])**2)
    return loss

def Z_norm(X):
    X_mean=X.mean()
    X_std=np.std(np.array(X))
    return (X-X_mean)/X_std, X_mean, X_std

def Z_norm_reverse(X,Xscaler,units_convert):
    return (X*Xscaler[1]+Xscaler[0])*units_convert

class R2Loss(nn.Module):
    #calculate coefficient of determination
    def forward(self, y_pred, y):
        var_y = torch.var(y, unbiased=False)
        return 1.0 - F.mse_loss(y_pred, y, reduction="mean") / var_y

In [9]:
target = 'NEE'

model = ['rcef_RandomForestRegressor', 'rcef_RidgeCV', 'rcef_XGBRegressor', 'xgboost'] 

extracted_features = model[-1]

In [10]:
# loads all data sets into a dict
def load_datasets(dirs: list, load_path: str) -> dict:
    files = ['soil_c','surf_water','flux_soc','soil_water','n_flux','p_flux','temp', 
         'plant_c','plant_n','plant_p','canopcy_c','plant_stress','photosynthesis','plant_growth']
    
    #files.append('soil_temp', 'canopy_temp') missing 
    datasets = {}
    
    for dr in dirs:
        csv_list = []
        path = 'datasets/' + dr + load_path
        for f in files:
            df = pd.read_csv(os.path.join(path,f + '.csv'))
            df.drop(df.columns[0], axis=1)

            csv_list.append(df)


        data_dict = {}
        for i in range (len(csv_list)):
            data_dict[files[i]] = csv_list[i]

        datasets[dr] = data_dict

    return datasets

dirs = ['warm_temp_maize_soybean_irrigated', 'warm_temp_maize-soybean_dryland', 'cool_temp_maize_soybean']

datasets = load_datasets(dirs, '/csv_outs/with_plant_soil_details/')

FileNotFoundError: [Errno 2] No such file or directory: 'datasets/warm_temp_maize_soybean_irrigated/csv_outs/with_plant_soil_details/soil_c.csv'

In [ ]:
def rename_dupes(suffix: str, df: pd.DataFrame, dupes: list) -> pd.DataFrame:
    for col in df.columns:
        if col in dupes:
            df.rename(columns={col: col + suffix}, inplace=True)
    return df

def average_numbered_columns(df):
    numbered_cols = [col for col in df.columns if '_' in col and col.split('_')[-1].isdigit()]

    col_groups = {}
    for col in numbered_cols:
        prefix = '_'.join(col.split('_')[:-1])
        if prefix not in col_groups:
            col_groups[prefix] = []
        col_groups[prefix].append(col)

    # calculate averages and add new columns
    for prefix, cols in col_groups.items():
        avg_col_name = prefix
        avg_col_values = df[cols].mean(axis=1)
        df[avg_col_name] = avg_col_values

    # drop numbered columns
    df = df.drop(columns=numbered_cols)

    return df

# turn all csv's to one dataframe
def to_pd(df: dict, handle_dupes: bool, flatten_num_cols: bool) -> pd.DataFrame:
    x = pd.DataFrame()
    for file_name in df:
        cur = df[file_name]
            
        x = pd.concat([x, df[file_name]], axis = 1)
        
    cheeky_col = 'unnamed.1'
    cheeky_col2 = 'Unnamed: 0'
    if cheeky_col in x.columns:
        x = x.drop([cheeky_col], axis=1)
    elif cheeky_col2 in x.columns:
         x = x.drop([cheeky_col2], axis=1)
    x = x.drop(['DATE'], axis=1)

    if flatten_num_cols:
        x = average_numbered_columns(x)
        
    x = x.loc[:,~x.columns.duplicated()].copy()
    
    one_hot = pd.get_dummies(x['GROWTH_STG'])
    x= x.drop('GROWTH_STG',axis = 1)
    # Join the encoded df
    x = x.join(one_hot)

    x.columns = x.columns.str.translate("".maketrans({"[":"{", "]":"}","<":"^"}))
    
    return x

df_dry = to_pd(datasets['warm_temp_maize-soybean_dryland'], True, True)
df_irr = to_pd(datasets['warm_temp_maize_soybean_irrigated'], True, True)
df_cool = to_pd(datasets['cool_temp_maize_soybean'], True, True)

# NEE = GPP - ER:
#GPP = GROSS PRIMARY PRODUCTION (TOTAL C INTAKE) 
#ER = total C uptake =  ECO_RH + ECO_RA =  autotrophic + heterotrophic respiration 
#NPP = GPP + ECO_RA
df_dry['NEE'] = df_dry['ECO_NPP'] - df_dry['ECO_RH']
df_irr['NEE'] = df_irr['ECO_NPP'] - df_irr['ECO_RH']
df_cool['NEE'] = df_cool['ECO_NPP'] - df_cool['ECO_RH']

df= pd.concat([df_dry, df_irr, df_cool])
y = df[target].copy()
#y = df[target].copy()
#df = df.drop(target, axis=1)

In [ ]:
xgbFeatImp = pd.read_csv('feature_analysis/xgboost/FeaturesImportance'  + target  + 'weather_soil_data' + '.csv')
feat_cols = []
for i in range(len(xgbFeatImp.values)):
    feat_cols.append(xgbFeatImp.values[i][0])

y = df[target].copy()
x = df[feat_cols]

In [ ]:
for entr in x.columns:
    x[entr] = preprocessing.normalize([x[entr]])[0]
y = pd.Series(preprocessing.normalize([y])[0], name='NEE')

### If Cuda available

In [ ]:
n_out=1
#shuffled_b=torch.randperm(X.size()[1]) # be aware that random may be different every time

##
#if torch.cuda.is_available():
#    device = torch.device("cuda")
#print(device)
##X=X[:,shuffled_b,:].to(device)   #test unshuffled site
##Y=Y[:,shuffled_b,:].to(device)
#X=df.to(device)  
#Y=y.to(device)
#print(X.size(),n_f)

train_n=70
val_n=10
test_n=19


X_train=X[:,0:train_n*fln,:].view(Tx*tyear,train_n*fln,n_f)
X_val=X[:,train_n*fln:(train_n+val_n)*fln,:].view(Tx*tyear,val_n*fln,n_f)
X_test=X[:,(train_n+val_n)*fln:(train_n+val_n+test_n)*fln,:].view(Tx*tyear,test_n*fln,n_f)
Y_train=Y[:,0:train_n*fln,:].view(Tx*tyear,train_n*fln,n_out)
Y_val=Y[:,train_n*fln:(train_n+val_n)*fln,:].view(Tx*tyear,val_n*fln,n_out)
Y_test=Y[:,(train_n+val_n)*fln:(train_n+val_n+test_n)*fln,:].view(Tx*tyear,test_n*fln,n_out)

#loss weights setup
loss_weights=[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0]   
print(X_train.size(), Y_train.size())

In [100]:
df['DATE'] = ( (df.index) % 365 ) 

In [101]:
X_train, X_test, Y_train, Y_test = train_test_split(
   df, y, test_size=0.15, random_state=41)

X_train, X_val, Y_train, Y_val = train_test_split(
    X_train, Y_train, test_size=0.15, random_state=1) #0.70 , 0.15 x 2

In [102]:
compute_r2=R2Loss()
n_a=64 #hidden state number
n_l=4 #layer of lstm
nout1=1
nout2=1
dropout=0.2
path_save = 'kgml-results'
os.makedirs(path_save, exist_ok=True)  
model1=KGML(len(features) + 1,n_a,n_l,1,dropout)
print("Model's state_dict:")
#model1.to(device)
print(model1)
params = list(model1.parameters())
print(len(params))
print(params[5].size())  # conv1's .weight

Model's state_dict:
KGML(
  (gru): GRU(16, 64, num_layers=4, dropout=0.2)
  (densor2): Linear(in_features=64, out_features=1, bias=True)
  (drop): Dropout(p=0.2, inplace=False)
)
18
torch.Size([192, 64])


In [119]:
starttime=time.time()
loss_weights=[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0] 
fln=15 #20 for full 0-300, 15 for 80-240
lr=0.1 #sgd
loss_val_best = 500000
lr_adam=0.0001
optimizer = optim.Adam(model1.parameters(), lr=lr_adam) #add weight decay normally 1-9e-4
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=600, gamma=0.5)
batch_total=X_train.shape[0]
batch_size=211  # this is the batch size for training
#during validation
train_losses = []
val_losses = []


batches = int(batch_total/batch_size)

#turn pd df to pytorch tensors
X_train_torch = torch.tensor(X_train.values)
Y_train_torch = torch.tensor(Y_train.values)
X_train_torch = X_train_torch.view(batches, batch_size, 16)
Y_train_torch = Y_train_torch.view(batches, batch_size, 1)

X_val_torch = torch.tensor(X_val.values)
Y_val_torch = torch.tensor(Y_val.values)

X_val_torch = X_val_torch.view(1, X_val_torch.size(0), X_val_torch.size(1))
Y_val_torch = Y_val_torch.view(Y_val_torch.size(0), 1, 1)

epoch=3000
model1.train()
for epoch in range(maxepoch):
    train_loss=0.0
    val_loss=0.0
    model1.zero_grad()
    
    
    Y_pred_all= torch.zeros(Y_train_torch.shape)
    
    for bb in range(int(batch_total/batch_size)):
        
        hidden = model1.init_hidden(batch_size)
        
        hidden = hidden.float()
    
        Y_pred,hidden = model1(X_train_torch[bb:bb+1, :].float(),\
                                hidden)
        
        loss = myloss_mul_sum(Y_pred, Y_train_torch[bb:(bb+1),:],\
                                 loss_weights)
        for nh in range(len(hidden[0])):
            hidden[0][nh].detach()
        hidden[1].detach()
        loss.backward()
        optimizer.step()
        with torch.no_grad():
            train_loss=train_loss+loss.item()
            Y_pred_all[bb:(bb+1),:]=Y_pred[:,:]
    scheduler.step()
    
    #validation
    model1.eval()
    with torch.no_grad():
        train_loss=train_loss/(batch_total/batch_size)
        train_losses.append(train_loss)
        train_R2=compute_r2(Y_pred_all[:,0].contiguous().view(-1),Y_train_torch[:,0].contiguous().view(-1)).item()
        Y_val_pred=torch.zeros(Y_train_torch.size())
        
        hidden = model1.init_hidden(X_val_torch.size(1))   
        Y_val_pred, hidden = model1(X_val_torch.float(), hidden)
        loss = myloss_mul_sum(Y_val_pred, Y_val_torch, loss_weights)
        val_loss=loss.item()
        val_losses.append(val_loss)
        #r2 only for n2o
        val_R2=compute_r2(Y_val_pred[:,:,0].contiguous().view(-1),Y_val_torch[:,:,0].contiguous().view(-1)).item()
        if val_loss < loss_val_best and val_R2 > R2_best:
            loss_val_best=val_loss
            R2_best = val_R2
            f0=open(path_save,'w')
            f0.close()
            #os.remove(path_save)
            torch.save({'epoch': epoch,
                    'model_state_dict': model1.state_dict(),
                    'R2': train_R2,
                    'loss': train_loss,
                    'los_val': val_loss,
                    'R2_val': val_R2,
                    }, path_save)    
        print("finished training epoch", epoch+1)
    mtime=time.time()
    print("train_loss: ", train_loss, "train_R2", train_R2)

    if train_R2 > 0.99:
        break
    model1.train()
endtime=time.time()
path_fs = path_save+'fs'
torch.save({'train_losses': train_losses,
            'val_losses': val_losses,
            'model_state_dict_fs': model1.state_dict(),
            }, path_fs)  
print("final train_loss:",train_loss,"final train_R2:",train_R2,"val_loss:",val_loss,"loss validation best:",)
print(f"total Training time: {endtime - starttime}s")

NameError: name 'loss_val_best' is not defined

In [198]:
for bb in range(int(batch_total/batch_size)):
    print(Y_train[bb*batch_size:(bb+1)*batch_size].shape)
    print("x train" + str(train_torch[bb*batch_size:(bb+1)*batch_size, :].shape))

(500,)
x traintorch.Size([500, 15])
(500,)
x traintorch.Size([500, 15])
(500,)
x traintorch.Size([500, 15])
(500,)
x traintorch.Size([500, 15])
(500,)
x traintorch.Size([500, 15])
(500,)
x traintorch.Size([500, 15])


In [161]:
Y_train[0:bb*batch_size:(bb+1)*batch_size]

Series([], Name: NEE, dtype: float64)

In [154]:
out_names = ['a','b','c']

In [156]:
out_names.index('b')

1